In [ ]:
from pandasql import sqldf
import pandas as pd
pysqldf = lambda q: sqldf(q, globals())


wr_data = pd.read_csv('wr_data.csv')
rb_data = pd.read_csv('rb_data.csv')
te_data = pd.read_csv('te_data.csv')
qb_data = pd.read_csv('qb_data.csv')

In [ ]:
# Finding the percent that return value by conference to see if its better to draft from a specific conference
# This cell is WRs, cell below is RBs

pysqldf("""
SELECT 
    Conference,  
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM wr_data
GROUP BY Conference
""")

In [ ]:
pysqldf("""
SELECT 
    Conference,  
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM rb_data
GROUP BY Conference
""")

In [ ]:
# Finding the average difference between position adp and ending rank based on those who return value and those who dont
# Essentially when they hit how much do they exceed expectations and when they miss how much do they miss expectations by
# This cell is for WRs, cell below is RBs

pysqldf("""
SELECT 
    CASE WHEN Returned_Value_Position = 1 THEN 'Returned Value' ELSE 'Did Not Return Value' END AS Return_Status,
    AVG(Position_ADP - Positional_Rank) AS Avg_Positional_Difference
FROM wr_data
GROUP BY Return_Status;
""")

In [ ]:
pysqldf("""
SELECT 
    CASE WHEN Returned_Value_Position = 1 THEN 'Returned Value' ELSE 'Did Not Return Value' END AS Return_Status,
    AVG(Position_ADP - Positional_Rank) AS Avg_Positional_Difference
FROM rb_data
GROUP BY Return_Status;
""")

In [ ]:
# Finding the % that return value based on how good the situation they are going into is

pysqldf("""
SELECT 
    CASE 
        WHEN Team_Drafted_to_Ws_Prev_Yr < 5 THEN 'Less than 5 Wins'
        WHEN Team_Drafted_to_Ws_Prev_Yr BETWEEN 5 AND 10 THEN '5-10 Wins'
        ELSE 'Over 10 Wins'
    END AS Wins_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM wr_data
GROUP BY Wins_Bucket
""")

In [ ]:
pysqldf("""
SELECT 
    CASE 
        WHEN Team_Drafted_to_Ws_Prev_Yr < 5 THEN 'Less than 5 Wins'
        WHEN Team_Drafted_to_Ws_Prev_Yr BETWEEN 5 AND 10 THEN '5-10 Wins'
        ELSE 'Over 10 Wins'
    END AS Wins_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM rb_data
GROUP BY Wins_Bucket
""")

In [ ]:
# Grouping value returning by reciever receptions

pysqldf("""
SELECT 
    CASE 
        WHEN Receptions < 60 THEN '<60 Receptions'
        WHEN Receptions BETWEEN 61 AND 75 THEN '61-75 Receptions'
        WHEN Receptions BETWEEN 76 AND 85 THEN '76-85 Receptions'
        WHEN Receptions BETWEEN 86 AND 100 THEN '86-100 Receptions'
        ELSE '100+ Receptions'
    END AS Receptions_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM wr_data
GROUP BY Receptions_Bucket
""")

In [ ]:

pysqldf("""
SELECT 
    CASE 
        WHEN Carries < 150 THEN '<150 Carries'
        WHEN Carries BETWEEN 151 AND 200 THEN '151-200 Carries'
        WHEN Carries BETWEEN 201 AND 250 THEN '201-250 Carries'
        WHEN Carries BETWEEN 251 AND 300 THEN '251-300 Carries'
        ELSE '300+ Yards'
    END AS Carries_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM rb_data
GROUP BY Carries_Bucket
""")

In [ ]:
# Grouping value returning by reciever yards

pysqldf("""
SELECT 
    CASE 
        WHEN Yards < 500 THEN '<500 Yards'
        WHEN Yards BETWEEN 500 AND 750 THEN '500-750 Yards'
        WHEN Yards BETWEEN 750 AND 1000 THEN '750-1000 Yards'
        WHEN Yards BETWEEN 1000 AND 1250 THEN '1000-1250 Yards'
        WHEN Yards BETWEEN 1250 AND 1500 THEN '1250-1500 Yards'
        ELSE '1500+ Yards'
    END AS Yards_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM wr_data
GROUP BY Yards_Bucket
""")

In [ ]:
pysqldf("""
SELECT 
    CASE 
        WHEN Yards < 1000 THEN '<1000 Yards'
        WHEN Yards BETWEEN 1000 AND 1300 THEN '1000-1300 Yards'
        WHEN Yards BETWEEN 750 AND 1000 THEN '750-1000 Yards'
        WHEN Yards BETWEEN 1301 AND 1600 THEN '1301-1600 Yards'
        WHEN Yards BETWEEN 1601 AND 2000 THEN '1601-2000 Yards'
        ELSE '2001+ Yards'
    END AS rb_Yards_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM rb_data
GROUP BY rb_Yards_Bucket
""")

In [ ]:
#Grouping value returning by reciver tds

pysqldf("""
SELECT 
    CASE 
        WHEN TDS BETWEEN 0 AND 5 THEN '0-5 TDs'
        WHEN TDS BETWEEN 6 AND 10 THEN '6-10 TDs'
        WHEN TDS BETWEEN 11 AND 15 THEN '11-15 TDs'
        WHEN TDS BETWEEN 16 AND 20 THEN '16-20 TDs'
        ELSE '20+ TDs'
    END AS TDs_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM wr_data
GROUP BY TDs_Bucket
""")

In [ ]:
pysqldf("""
SELECT 
    CASE 
        WHEN TDS BETWEEN 1 AND 9 THEN '1-9 TDs'
        WHEN TDS BETWEEN 10 AND 14 THEN '10-14 TDs'
        WHEN TDS BETWEEN 15 AND 19 THEN '15-19 TDs'
        WHEN TDS BETWEEN 20 AND 30 THEN '20-30 TDs'
        ELSE '20+ TDs'
    END AS rb_TDs_Bucket,
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM rb_data
GROUP BY rb_TDs_Bucket
""")

In [ ]:
#Grouping by draft round

pysqldf("""
SELECT 
    Draft_Round
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM wr_data
GROUP BY Round
""")

In [ ]:
pysqldf("""
SELECT 
    Draft_Round
    AVG(Returned_Value_Position) * 100 AS Avg_Returned_Value_Percentage
FROM rb_data
GROUP BY Round
""")

In [ ]:
#Finding the players who exceeded expectations, those who finished 25% above expectations

pysqldf("""
SELECT *
FROM wr_data
WHERE Positional_Rank <= Position_ADP * 0.75;
""")

In [ ]:
pysqldf("""
SELECT *
FROM rb_data
WHERE Positional_Rank <= Position_ADP * 0.75;
""")

In [ ]:
#Finding the players who REALLY exceeded expectations, those who finished 50% above expectations
pysqldf("""
SELECT *
FROM wr_data
WHERE Positional_Rank <= Position_ADP * 0.5;
""")

In [ ]:
pysqldf("""
SELECT *
FROM rb_data
WHERE Positional_Rank <= Position_ADP * 0.5;
""")